In [102]:
import os
import time
import csv
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR

from typing import Optional

In [103]:
WINDOW_SIZE = 64
BATCH_SIZE = 64
q=3

In [104]:
def calc(pred, true):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for pre, gt in zip(pred, true):
        if gt == 1:
            if pre == 1:
                TP += 1
            else:
                FN += 1
        if gt == 0:
            if pre == 1:
                FP += 1
            else:
                TN += 1
    #print('TP=%d FP=%d TN=%d FN=%d' % (TP, FP, TN, FN))
    return TP, FP, TN, FN

class DataGenerator:
    """
    to inject anomalous points according to the formula in the paper:
    """
    def __init__(self, win_siz, step, nums=5):
        self.win_size = win_siz
        self.step = step
        self.number = nums

    def generate_train_data(self, values, labels, insert_anomaly=False):
        def normalize(a):
            amin = np.min(a)
            amax = np.max(a)
            a = (a - amin) / (amax - amin + 1e-5)
            return 3 * a

        back=5
        length = len(values)
        tmp = []
        for pt in range(self.win_size, length - back, self.step):
            head = max(0, pt - self.win_size)
            tail = min(length - back, pt)
            data = np.array(values[head:tail])
            data = data.astype(np.float64)
            data = normalize(data)
            label = np.array(labels[head:tail])
            if insert_anomaly:
                num = np.random.randint(1, self.number)
                ids = np.random.choice(self.win_size, num, replace=False)
                lbs = np.zeros(self.win_size, dtype=np.int64)
                if (self.win_size - 6) not in ids:
                    self.control += np.random.random()
                else:
                    self.control = 0
                if self.control > 100:
                    ids[0] = self.win_size - 6
                    self.control = 0
                mean = np.mean(data)
                dataavg = average_filter(data)
                var = np.var(data)
                for id in ids:
                    data[id] += (dataavg[id] + mean) * np.random.randn() * min((1 + var), 10)
                    label[id] = 1
            tmp.append([data.tolist(), label.tolist()])
        return tmp

def average_filter(values, n=3):
    """
    Calculate the sliding window average for the give time series.
    Mathematically, res[i] = sum_{j=i-t+1}^{i} values[j] / t, where t = min(n, i+1)
    :param values: list.
        a list of float numbers
    :param n: int, default 3.
        window size.
    :return res: list.
        a list of value after the average_filter process.
    """

    if n >= len(values):
        n = len(values)

    res = np.cumsum(values, dtype=float)
    res[n:] = res[n:] - res[:-n]
    res[n:] = res[n:] / n

    for i in range(1, n):
        res[i] /= (i + 1)

    return res

def spectral_residual(values, n=q):
    """
    This method transform a time series into spectral residual series
    :param values: list.
        a list of float values.
    :return: mag: list.
        a list of float values as the spectral residual values
    """
    EPS = 1e-8
    trans = np.fft.fft(values)
    mag = np.sqrt(trans.real ** 2 + trans.imag ** 2)
    eps_index = np.where(mag <= EPS)[0]
    mag[eps_index] = EPS

    mag_log = np.log(mag)
    mag_log[eps_index] = 0

    spectral = np.exp(mag_log - average_filter(mag_log, n))

    trans.real = trans.real * spectral / mag
    trans.imag = trans.imag * spectral / mag
    trans.real[eps_index] = 0
    trans.imag[eps_index] = 0

    wave_r = np.fft.ifft(trans)
    mag = np.sqrt(wave_r.real ** 2 + wave_r.imag ** 2)
    return mag


def load_kpi(csv_path):
    kpis = {}
    anomalies = 0
    with open(csv_path) as f:
        input = csv.reader(f, delimiter=',')
        cnt = 0
        for row in input:
            if cnt == 0:
                cnt += 1
                continue
            kpi = kpis.get(str(row[3]), [[], [], []])
            kpi[0].append(int(row[0]))  # timestamp
            kpi[1].append(float(row[1]))  # value
            kpi[2].append(int(row[2]))  # label
            kpis[str(row[3])] = kpi
            cnt += 1
            if int(row[2]) == 1:
                anomalies += 1
        print("Training data loaded. Total length: {}, number of anomalies: {}".format(cnt, anomalies))
        f.close()
    return kpis


class DatasetIterator(Dataset):
    def __init__(self, width, data, label_anomalies=False):
        self.genlen = 0
        self.len = self.genlen
        self.width = width
        self.kpinegraw = data
        self.negrawlen = len(self.kpinegraw)
        print('length :', len(self.kpinegraw))
        self.len += self.negrawlen
        self.kpineglen = 0
        self.control = 0.
        self.label_anomaly = label_anomalies

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        if isinstance(index, slice):
            start = index.start if index.start else 0
            end = index.stop if index.stop else self.len
            return [self.__getitem(idx) for idx in range(start, end)]
        
        else:
            return self.__getitem(index)
    
    def __getitem(self, index):
        idx = index % self.negrawlen
        datas = self.kpinegraw[idx]
        datas = np.array(datas)
        data = datas[0, :].astype(np.float64)
        lbs = datas[1, :].astype(np.float64)
        if self.label_anomaly:
            wave = spectral_residual(data)
            waveavg = average_filter(wave, n=z)
            for i in range(self.width):
                if wave[i] < 0.001 and waveavg[i] < 0.001:
                    lbs[i] = 0
                    continue
                ratio = wave[i] / waveavg[i]
                if ratio < 1.0 and lbs[i] == 1:
                    lbs[i] = 0
                if ratio > 5.0:
                    lbs[i] = 1
            srscore = abs(wave - waveavg) / (waveavg + 0.001)
            data = wave
            lbs[srscore > TAU] = 1
        resdata = torch.from_numpy(data)
        reslb = torch.from_numpy(lbs)
        return resdata, reslb

def highlight(indices,ax, thickness=0.5):
    i=0
    while i<len(indices):
        ax.axvspan(indices[i]-thickness, indices[i]+thickness, facecolor='red', edgecolor='none', alpha=.9)
        i+=1

def plots(values, is_anomaly, interval=[0,100], fig_size=(40,4), thickness=0.5):
    df = pd.DataFrame()
    start_index= (len(values) * interval[0]) // 100
    end_index = (len(values) * interval[1]) // 100
    df['value'] = values[start_index:end_index]
    df['is_anomaly'] = is_anomaly[start_index:end_index]
    df = df.reset_index()
    
    ax1 = df.plot(kind='line', x='index', y='value', color='b', figsize=fig_size) 
    highlight(df[df['is_anomaly']==1].index, ax1, thickness)


In [105]:
training_data_location = os.getcwd()+'/../srcnn/training_data/'

In [ ]:
kpis = {}
DATA_PATH=os.getcwd()+'/../srcnn/train.csv'
anomalies = 0
with open(DATA_PATH) as f:
    input = csv.reader(f, delimiter=',')
    cnt = 0
    for row in input:
        if cnt == 0:
            cnt += 1
            continue
        kpi = kpis.get(str(row[3]), [[],[],[]])
        kpi[0].append(int(row[0])) #timestamp
        kpi[1].append(float(row[1])) #value
        kpi[2].append(int(row[2])) #label
        kpis[str(row[3])] = kpi
        cnt += 1
        if int(row[2]) == 1:
            anomalies += 1
    f.close()

In [25]:
import csv
i=1
header = ['timestamp', 'value', 'label']
training_data_location = os.getcwd()+'/../srcnn/training_data/'
for kpi in kpis.values():
    with open(training_data_location+"series"+str(i)+".csv", 'w', encoding='UTF8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        for idx in range(0,len(kpi[0])):
            writer.writerow({header[0]: kpi[0][idx], header[1]: kpi[1][idx], header[2]: kpi[2][idx]})
    i+=1
        

NameError: name 'kpis' is not defined

In [106]:
import glob
all_files = glob.glob(training_data_location+"/*.csv")
print("Total number of files: ", len(all_files))

Total number of files:  396


In [108]:
minimum=99999999
kpi_dataset = glob.glob(training_data_location+"/series*.csv")
lengths = []
all_files = [item for item in all_files if item not in kpi_dataset]
#for filename in filenames:
#    file = open(filename)
#    reader = csv.reader(file)
#    lines= len(list(reader))
#    filenames.append(filenam)


In [109]:
from sklearn.model_selection import train_test_split
seed = 12345
training_files, test_files = train_test_split(all_files, test_size = 0.20, random_state = seed)
training_files, validation_files = train_test_split(training_files, test_size = 0.10, random_state = seed)

In [110]:
len(training_files)

263

In [111]:
len(test_files)

74

In [112]:
len(validation_files)

30

In [113]:
np.random.shuffle(training_files)

In [157]:
def get_data(filenames):
    counts = [0,0]
    result = [[],[]]
    for file in tqdm(filenames):
        names = ['timestamp','value','label']
        df = pd.read_csv(file, header=None, skipinitialspace=True, usecols=[0,1,2], names=names, skiprows=[0])
        for idx in range(0, len(df) - WINDOW_SIZE):
            head = idx
            tail = idx+WINDOW_SIZE
            data = spectral_residual(df['value'].fillna(0)[head:tail].to_numpy())
            labels = df['label'].fillna(0)[head:tail].to_numpy()
            output = labels[WINDOW_SIZE-1]
            counts[output] += 1
            count_arr = np.bincount(labels)
            #if 1 not in labels:
            #    continue
            if len(count_arr) <= 1 or count_arr[0] < count_arr[1]:
                continue
            result[output].append((data, output))
    print('zero outputs: ', counts[0], ' one outputs: ', counts[1])
    return result


In [158]:
training_dataset = get_data(filenames = training_files)
validation_dataset = get_data(filenames = validation_files)
test_dataset = get_data(filenames = test_files)

  0%|          | 0/263 [00:00<?, ?it/s]

zero outputs:  390971  one outputs:  2705


  0%|          | 0/30 [00:00<?, ?it/s]

zero outputs:  43613  one outputs:  233


  0%|          | 0/74 [00:00<?, ?it/s]

zero outputs:  111069  one outputs:  887


In [159]:
i=0

In [125]:
while 1 not in training_dataset[i][1]:
    i += 1
plots(training_dataset[i][0], training_dataset[i][1])
i+=1

TypeError: argument of type 'numpy.int64' is not iterable

In [126]:
train_dataloader = DataLoader(training_dataset, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
val_dataloader = DataLoader(validation_dataset, batch_size = BATCH_SIZE, drop_last=True) # Shuffle is False by default.
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, drop_last=True)

In [127]:
class AbsPool(nn.Module):
    def __init__(self, pooling_module=None, *args, **kwargs):
        super(AbsPool, self).__init__()
        self.pooling_layer = pooling_module(*args, **kwargs)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        pos_pool = self.pooling_layer(x)
        neg_pool = self.pooling_layer(-x)
        abs_pool = torch.where(pos_pool >= neg_pool, pos_pool, -neg_pool)
        return abs_pool


MaxAbsPool1d = partial(AbsPool, nn.MaxPool1d)
MaxAbsPool2d = partial(AbsPool, nn.MaxPool2d)
MaxAbsPool3d = partial(AbsPool, nn.MaxPool3d)

loss_results = []
accuracy_results = []
accuracy_histories = []


In [128]:
class SharpenedCosineSimilarity(nn.Conv1d):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int=3,
        padding: int='same',
        stride: int=1,
        groups: int=1,
        shared_weights: bool = False,
        log_p_init: float=.7,
        log_q_init: float=1.,
        log_p_scale: float=5.,
        log_q_scale: float=.3,
        alpha: Optional[float]=10,
        alpha_autoinit: bool=False,
        eps: float=1e-6,
    ):
        assert groups == 1 or groups == in_channels, " ".join([
            "'groups' needs to be 1 or 'in_channels' ",
            f"({in_channels})."])
        assert out_channels % groups == 0, " ".join([
            "The number of",
            "output channels needs to be a multiple of the number",
            "of groups.\nHere there are",
            f"{out_channels} output channels and {groups}",
            "groups."])

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.groups = groups
        self.shared_weights = shared_weights

        if self.groups == 1:
            self.shared_weights = False

        super(SharpenedCosineSimilarity, self).__init__(
            self.in_channels,
            self.out_channels,
            kernel_size,
            bias=False,
            padding=padding,
            stride=stride,
            groups=self.groups)

        # Overwrite self.kernel_size created in the 'super' above.
        # We want an int, assuming a square kernel, rather than a tuple.
        self.kernel_size = kernel_size

        # Scaling weights in this way generates kernels that have
        # an l2-norm of about 1. Since they get normalized to 1 during
        # the forward pass anyway, this prevents any numerical
        # or gradient weirdness that might result from large amounts of
        # rescaling.
        self.channels_per_kernel = self.in_channels // self.groups
        weights_per_kernel = self.channels_per_kernel * self.kernel_size ** 2
        if self.shared_weights:
            self.n_kernels = self.out_channels // self.groups
        else:
            self.n_kernels = self.out_channels
        initialization_scale = (3 / weights_per_kernel) ** .5
        scaled_weight = np.random.uniform(
            low=-initialization_scale,
            high=initialization_scale,
            size=(
                self.n_kernels,
                self.channels_per_kernel,
                self.kernel_size)
        )
        self.weight = torch.nn.Parameter(torch.Tensor(scaled_weight))

        self.log_p_scale = log_p_scale
        self.log_q_scale = log_q_scale
        self.p = torch.nn.Parameter(torch.full(
            (1, self.n_kernels, 1),
            float(log_p_init * self.log_p_scale)))
        self.q = torch.nn.Parameter(torch.full(
            (1, 1, 1), float(log_q_init * self.log_q_scale)))
        self.eps = eps

        if alpha is not None:
            self.alpha = torch.nn.Parameter(torch.full(
                (1, 1, 1), float(alpha)))
        else:
            self.alpha = None
        if alpha_autoinit and (alpha is not None):
            self.LSUV_like_init()

    def LSUV_like_init(self, batch_size=BATCH_SIZE):
        BS, CH = batch_size, int(self.weight.shape[1]*self.groups)
        L = self.weight.shape[2]
        device = self.weight.device
        inp = torch.rand(BS, CH, L, device=device)
        with torch.no_grad():
            out = self.forward(inp)
            coef = (out.std(dim=(0, 2)) + self.eps).mean()
            self.alpha.data *= 1.0 / coef.view_as(self.alpha)
        return

    def forward(self, inp: torch.Tensor) -> torch.Tensor:
        # Scale and transform the p and q parameters
        # to ensure that their magnitudes are appropriate
        # and their gradients are smooth
        # so that they will be learned well.
        p = torch.exp(self.p / self.log_p_scale)
        q = torch.exp(-self.q / self.log_q_scale)

        # If necessary, expand out the weight and p parameters.
        if self.shared_weights:
            weight = torch.tile(self.weight, (self.groups, 1, 1))
            p = torch.tile(p, (1, self.groups, 1))
        else:
            weight = self.weight

        return self.scs(inp, weight, p, q)

    def scs(self, inp, weight, p, q):
        # Normalize the kernel weights.
        weight = weight / self.weight_norm(weight)

        # Normalize the inputs and
        # Calculate the dot product of the normalized kernels and the
        # normalized inputs.
        cos_sim = F.conv1d(
            inp,
            weight,
            stride=self.stride,
            padding=self.padding,
            groups=self.groups,
        ) / self.input_norm(inp, q)

        # Raise the result to the power p, keeping the sign of the original.
        out = cos_sim.sign() * (cos_sim.abs() + self.eps) ** p

        # Apply learned scale parameter
        if self.alpha is not None:
            out = self.alpha.view(1, -1, 1) * out
        return out

    def weight_norm(self, weight):
        # Find the l2-norm of the weights in each kernel.
        return weight.square().sum(dim=(1, 2), keepdim=True).sqrt()

    def input_norm(self, inp, q):
        # Find the l2-norm of the inputs at each position of the kernels.
        # Sum the squared inputs over each set of kernel positions
        # by convolving them with the mock all-ones kernel weights.
        xnorm = F.conv1d(
            inp.square(),
            torch.ones((
                self.groups,
                self.channels_per_kernel,
                self.kernel_size)),
            stride=self.stride,
            padding=self.padding,
            groups=self.groups)

        # Add in the q parameter. 
        xnorm = (xnorm + self.eps).sqrt() + q
        outputs_per_group = self.out_channels // self.groups
        return torch.repeat_interleave(xnorm, outputs_per_group, axis=1)

In [145]:
n_input_channels = 1
n_units = 64
kernel_size = 7

class Network(nn.Module):
    def __init__(self, window_size=WINDOW_SIZE):
        super().__init__()
        self.window_size = window_size
        self.scs1 = SharpenedCosineSimilarity(
            in_channels=n_input_channels,
            out_channels=n_units,
            kernel_size=kernel_size,
            groups=n_input_channels)
        self.pool1 = MaxAbsPool1d(kernel_size=2, stride=2, ceil_mode=True)

        self.scs2_depth = SharpenedCosineSimilarity(
            in_channels=n_units,
            out_channels=n_units,
            kernel_size=kernel_size,
            groups=n_units)
        self.scs2_point = SharpenedCosineSimilarity(
            in_channels=n_units,
            out_channels=n_units,
            kernel_size=1)
        self.pool2 = MaxAbsPool1d(kernel_size=2, stride=2, ceil_mode=True)

        self.scs3_depth = SharpenedCosineSimilarity(
            in_channels=n_units,
            out_channels=n_units,
            kernel_size=kernel_size,
            groups=n_units,
            padding=1)
        self.scs3_point = SharpenedCosineSimilarity(
            in_channels=n_units,
            out_channels=n_units,
            kernel_size=1)
        self.pool3 = MaxAbsPool1d(kernel_size=4, stride=4, ceil_mode=True)

        self.scs4_depth = SharpenedCosineSimilarity(
            in_channels=n_units,
            out_channels=n_units,
            kernel_size=kernel_size,
            groups=n_units,
            padding=1)
        self.scs4_point = SharpenedCosineSimilarity(
            in_channels=n_units,
            out_channels=n_units,
            kernel_size=1)
        self.pool4 = MaxAbsPool1d(kernel_size=4, stride=4, ceil_mode=True)
        
        self.scs5_depth = SharpenedCosineSimilarity(
            in_channels=n_units,
            out_channels=n_input_channels,
            kernel_size=kernel_size,
            groups=n_input_channels)
        self.scs5_point = SharpenedCosineSimilarity(
            in_channels=n_input_channels,
            out_channels=n_input_channels,
            kernel_size=1)
        
        self.out = nn.Linear(in_features=n_units, out_features=self.window_size)
        self.out2 = nn.Linear(in_features=n_units, out_features=n_units*2)
        self.out3 = nn.Linear(in_features=n_units*2, out_features=1)

    def forward(self, t):
        t = t.reshape(t.size(0), n_input_channels, -1)
        #print('0 t.shape:', t.shape)
        t = self.scs1(t)
        #t = self.pool1(t)
        #print('1 t.shape:', t.shape)
        t = self.scs2_depth(t)
        #print('2 t.shape:', t.shape)
        t = self.scs2_point(t)
        #t = self.pool2(t)

        #t = self.scs3_depth(t)
        #t = self.scs3_point(t)
        #t = self.pool3(t)

        #t = self.scs4_depth(t)
        #t = self.scs4_point(t)
        #t = self.pool4(t)
        
        #t = t.reshape(-1, n_units)
        #t = self.out(t)
        t = self.scs5_depth(t)
        #print('2.1 t.shape:', t.shape)
        t = self.scs5_point(t)
        
        t = t.reshape(t.size(0), -1)
        #print('3 t.shape:', t.shape)
        t = self.out2(t)
        #print('4 t.shape:', t.shape)
        t = self.out3(t)
        #print('5 t.shape:', t.shape)
        return torch.sigmoid(t)

In [151]:
max_lr = .5
steps_per_epoch = len(train_dataloader)
n_epochs = 10

network = Network()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
network = network.to(device).float()

#for p in network.parameters():
#    if p.requires_grad:
#        print(p.numel())

n_params = sum(p.numel() for p in network.parameters() if p.requires_grad)
print(f"Model has {n_params:_} trainable parameters.")

optimizer = optim.Adam(network.parameters(), lr=max_lr)
scheduler = OneCycleLR(
    optimizer,
    max_lr=max_lr,
    steps_per_epoch=steps_per_epoch,
    epochs=n_epochs)

epoch_accuracy_history = []
for i_epoch in range(n_epochs):
    totalTP, totalFP, totalTN, totalFN = 0,0,0,0
    epoch_start_time = time.time()
    epoch_training_loss = 0
    epoch_testing_loss = 0
    epoch_training_num_correct = 0
    epoch_testing_num_correct = 0

    with tqdm(enumerate(train_dataloader)) as tqdm_training_loader:
        for batch_idx, batch in tqdm_training_loader:

            series, labels = batch
            series, labels = series.to(device), labels.to(device)
            #print('series input shape:', series.shape)
            #print('labels input shape:', labels.shape)
            #print('kucuk data?', series.shape)
            if len(series) < WINDOW_SIZE:
                #print('kucuk data!!')
                continue

            preds = network(series.float())
            #print('preds shape:', preds.shape)
            #print('labels shape:', labels.reshape(-1, 1).shape)
            loss = nn.BCELoss()(preds.reshape(-1, 1), labels.reshape(-1, 1).float())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            epoch_training_loss += loss.item() * train_dataloader.batch_size
            preds[preds < 0.5] = 0
            preds[preds >= 0.5] = 1
            TP, FP, TN, FN =  calc(preds.detach().cpu().numpy().reshape(-1), labels.detach().cpu().numpy().reshape(-1))
            totalTP += TP
            totalFP += FP
            totalTN += TN
            totalFN += FN
            print(TP, FP, TN, FN)
            epoch_training_num_correct += (
                preds.eq(labels).sum().item()) * 1. / len(labels)

            tqdm_training_loader.set_description(
                f'Step: {batch_idx + 1}/{steps_per_epoch}, '
                f'Epoch: {i_epoch + 1}/{n_epochs}'
            )

    epoch_duration = time.time() - epoch_start_time
    training_loss = epoch_training_loss / len(train_dataloader.dataset)
    training_accuracy = (
        epoch_training_num_correct / len(train_dataloader.dataset))
    training_precision = totalTP * 1.0 / (totalTP+totalFP+1e-10)
    training_recall =  totalTP/(totalTP+totalFN+1e-10)
    print(f"training_precision: {training_precision}   ")
    print(f"training_recall: {training_recall}   ")
    totalTP, totalFP, totalTN, totalFN = 0,0,0,0
    # At the end of each epoch run the testing data through an
    # evaluation pass to see how the model is doing.
    # Specify no_grad() to prevent a nasty out-of-memory condition.
    with torch.no_grad():
        for batch in test_dataloader:
            series, labels = batch
            series, labels = series.to(device), labels.to(device)
            #print('kucuk data?', len(series))
            if len(series) < WINDOW_SIZE:
                #print('kucuk data!!')
                continue
            preds = network(series.float())
            #print('preds ', preds.shape)
            #print('labels ', labels.shape)
            loss = nn.BCELoss()(preds.reshape(-1, 1), labels.reshape(-1, 1).float())

            epoch_testing_loss += loss.item() * test_dataloader.batch_size
            preds[preds < 0.5] = 0
            preds[preds >= 0.5] = 1
            epoch_testing_num_correct += (
                preds.eq(labels).sum().item()) * 1. / len(labels)
            TP, FP, TN, FN = calc(preds.detach().cpu().numpy().reshape(-1), labels.detach().cpu().numpy().reshape(-1))
            totalTP += TP
            totalFP += FP
            totalTN += TN
            totalFN += FN

        testing_loss = epoch_testing_loss / len(test_dataloader.dataset)
        testing_accuracy = (
            epoch_testing_num_correct / len(test_dataloader.dataset))
        epoch_accuracy_history.append(testing_accuracy)

    print(
        f"epoch: {i_epoch}   "
        f"duration: {epoch_duration:.04}   "
        f"learning rate: {scheduler.get_last_lr()[0]:.04}   "
        f"training loss: {training_loss:.04}   "
        f"testing loss: {testing_loss:.04}   "
        f"training accuracy: {100 * training_accuracy:.04}%   "
        f"testing accuracy: {100 * testing_accuracy:.04}%"
    )
    testing_precision = totalTP * 1.0 / (totalTP+totalFP+1e-10)
    testing_recall =  totalTP/(totalTP+totalFN+1e-10)
    print(f"testing_precision: {testing_precision}   ")
    print(f"testing_recall: {testing_recall}   ")
accuracy_histories.append(epoch_accuracy_history)
accuracy_results.append(testing_accuracy)
loss_results.append(testing_loss)

Model has 27_606 trainable parameters.


0it [00:00, ?it/s]

2 54 8 0
0 0 64 0
0 0 62 2
0 0 64 0
0 0 63 1
0 0 62 2
0 0 63 1
0 0 62 2
0 0 62 2
0 0 63 1
0 0 62 2
0 0 60 4
0 0 63 1
0 0 63 1
0 0 63 1
0 0 62 2
0 0 62 2
0 0 62 2
0 0 63 1
0 0 63 1
0 0 62 2
0 0 64 0
0 0 61 3
0 0 62 2
0 0 61 3
0 0 60 4
0 0 62 2
0 0 63 1
0 0 63 1
0 0 62 2
0 0 62 2
0 0 61 3
0 0 61 3
0 0 64 0
0 0 63 1
2 62 0 0
3 61 0 0
0 64 0 0
0 0 57 7
0 0 60 4
0 0 62 2
0 0 62 2
0 0 60 4
0 0 64 0
0 0 64 0
0 0 63 1
0 0 62 2
0 0 63 1
0 0 62 2
0 0 63 1
0 0 64 0
0 0 61 3
0 0 61 3
0 0 60 4
0 0 62 2
0 0 61 3
0 0 62 2
0 0 62 2
0 0 64 0
0 0 64 0
0 0 61 3
0 0 63 1
0 0 64 0
0 0 62 2
0 0 62 2
0 0 62 2
0 0 63 1
0 0 62 2
0 0 64 0
0 0 62 2
0 0 62 2
0 0 63 1
0 0 64 0
0 0 64 0
0 0 63 1
0 0 63 1
0 0 57 7
0 0 60 4
0 0 63 1
0 0 61 3
0 0 62 2
0 0 61 3
0 0 62 2
0 0 62 2
0 0 63 1
0 0 62 2
0 0 62 2
0 0 62 2
0 0 62 2
0 0 61 3
0 0 63 1
0 0 64 0
0 0 63 1
0 0 59 5
0 0 62 2
0 0 63 1
0 0 64 0
0 0 62 2
0 0 61 3
0 0 63 1
0 0 62 2
0 0 64 0
0 0 61 3
0 0 61 3
0 0 62 2
0 0 61 3
0 0 63 1
0 0 59 5
0 0 62 2
0 0 63 1
0 0 62 2
0

0 0 62 2
0 0 56 8
0 0 63 1
0 0 62 2
0 0 62 2
0 0 62 2
0 0 60 4
0 0 63 1
0 0 60 4
0 0 62 2
0 0 64 0
0 0 61 3
0 0 62 2
0 0 61 3
0 0 62 2
0 0 61 3
0 0 61 3
0 0 64 0
0 0 64 0
0 0 63 1
0 0 64 0
0 0 63 1
0 0 60 4
0 0 60 4
0 0 60 4
0 0 60 4
0 0 62 2
0 0 60 4
0 0 63 1
0 0 60 4
0 0 64 0
0 0 62 2
0 0 63 1
0 0 63 1
0 0 61 3
0 0 63 1
0 0 62 2
0 0 63 1
0 0 61 3
0 0 63 1
0 0 64 0
0 0 63 1
0 0 62 2
0 0 59 5
0 0 63 1
0 0 64 0
0 0 63 1
0 0 61 3
0 0 63 1
0 0 63 1
0 0 62 2
0 0 62 2
0 0 62 2
0 0 59 5
0 0 63 1
0 0 62 2
0 0 62 2
0 0 63 1
0 0 62 2
0 0 63 1
0 0 62 2
0 0 62 2
0 0 64 0
0 0 61 3
0 0 64 0
0 0 63 1
0 0 62 2
0 0 62 2
0 0 62 2
0 0 61 3
0 0 64 0
0 0 61 3
0 0 63 1
0 0 64 0
0 0 64 0
0 0 64 0
0 0 59 5
0 0 64 0
0 0 63 1
0 0 64 0
0 0 64 0
0 0 64 0
0 0 62 2
0 0 61 3
0 0 63 1
0 0 62 2
0 0 63 1
0 0 62 2
0 0 61 3
0 0 62 2
0 0 60 4
0 0 64 0
0 0 62 2
0 0 62 2
0 0 63 1
0 0 61 3
0 0 58 6
0 0 63 1
0 0 63 1
0 0 63 1
0 0 62 2
0 0 63 1
0 0 64 0
0 0 62 2
0 0 62 2
0 0 63 1
0 0 61 3
0 0 64 0
0 0 62 2
0 0 62 2
0 0 61 3
0

0it [00:00, ?it/s]

0 0 62 2
0 0 64 0
0 0 63 1
0 0 61 3
0 0 60 4
0 0 62 2
0 0 63 1
0 0 63 1
0 0 62 2
0 0 63 1
0 0 60 4
0 0 64 0
0 0 63 1
0 0 61 3
0 0 63 1
0 0 62 2
0 0 62 2
0 0 63 1
0 0 62 2
0 0 62 2
0 0 62 2
0 0 63 1
0 0 60 4
0 0 61 3
0 0 63 1
0 0 60 4
0 0 61 3
0 0 61 3
0 0 62 2
0 0 62 2
0 0 62 2
0 0 62 2
0 0 62 2
0 0 64 0
0 0 61 3
0 0 62 2
0 0 62 2
0 0 63 1
0 0 63 1
0 0 61 3
0 0 61 3
0 0 61 3
0 0 61 3
0 0 62 2
0 0 62 2
0 0 62 2
0 0 61 3
0 0 60 4
0 0 63 1
0 0 63 1
0 0 61 3
0 0 62 2
0 0 61 3
0 0 61 3
0 0 62 2
0 0 64 0
0 0 62 2
0 0 64 0
0 0 64 0
0 0 62 2
0 0 64 0
0 0 63 1
0 0 61 3
0 0 62 2
0 0 63 1
0 0 58 6
0 0 61 3
0 0 62 2
0 0 60 4
0 0 62 2
0 0 61 3
0 0 63 1
0 0 64 0
0 0 60 4
0 0 62 2
0 0 58 6
0 0 63 1
0 0 62 2
0 0 64 0
0 0 64 0
0 0 59 5
0 0 62 2
0 0 64 0
0 0 60 4
0 0 62 2
0 0 62 2
0 0 63 1
0 0 62 2
0 0 59 5
0 0 62 2
0 0 60 4
0 0 64 0
0 0 61 3
0 0 61 3
0 0 62 2
0 0 63 1
0 0 63 1
0 0 61 3
0 0 63 1
0 0 61 3
0 0 61 3
0 0 62 2
0 0 62 2
0 0 62 2
0 0 64 0
0 0 63 1
0 0 62 2
0 0 61 3
0 0 63 1
0 0 60 4
0 0 64 0
0

0 0 64 0
0 0 63 1
0 0 62 2
0 0 61 3
0 0 61 3
0 0 63 1
0 0 63 1
0 0 63 1
0 0 63 1
0 0 62 2
0 0 64 0
0 0 62 2
0 0 63 1
0 0 63 1
0 0 62 2
0 0 61 3
0 0 64 0
0 0 63 1
0 0 62 2
0 0 63 1
0 0 62 2
0 0 63 1
0 0 62 2
0 0 60 4
0 0 62 2
0 0 64 0
0 0 62 2
0 0 61 3
0 0 63 1
0 0 63 1
0 0 62 2
0 0 62 2
0 0 64 0
0 0 61 3
0 0 58 6
0 0 63 1
0 0 64 0
0 0 61 3
0 0 63 1
0 0 64 0
0 0 63 1
0 0 61 3
0 0 63 1
0 0 64 0
0 0 60 4
0 0 64 0
0 0 63 1
0 0 63 1
0 0 64 0
0 0 62 2
0 0 61 3
0 0 64 0
0 0 61 3
0 0 62 2
0 0 63 1
0 0 61 3
0 0 62 2
0 0 61 3
0 0 61 3
0 0 63 1
0 0 61 3
0 0 62 2
0 0 61 3
0 0 63 1
0 0 62 2
0 0 64 0
0 0 64 0
0 0 62 2
0 0 61 3
0 0 64 0
0 0 63 1
0 0 64 0
0 0 59 5
0 0 64 0
0 0 59 5
0 0 64 0
0 0 59 5
0 0 61 3
0 0 63 1
0 0 63 1
0 0 60 4
0 0 62 2
0 0 63 1
0 0 58 6
0 0 62 2
0 0 62 2
0 0 61 3
0 0 62 2
0 0 60 4
0 0 64 0
0 0 62 2
0 0 60 4
0 0 63 1
0 0 62 2
0 0 61 3
0 0 60 4
0 0 62 2
0 0 62 2
0 0 64 0
0 0 62 2
0 0 58 6
0 0 63 1
0 0 60 4
0 0 64 0
0 0 60 4
0 0 64 0
0 0 63 1
0 0 62 2
0 0 63 1
0 0 64 0
0 0 57 7
0

0it [00:00, ?it/s]

0 0 60 4
0 0 62 2
0 0 62 2
0 0 64 0
0 0 63 1
0 0 62 2
0 0 62 2
0 0 60 4
0 0 62 2
0 0 63 1
0 0 64 0
0 0 64 0
0 0 63 1
0 0 61 3
0 0 61 3
0 0 61 3
0 0 62 2
0 0 62 2
0 0 61 3
0 0 60 4
0 0 62 2
0 0 61 3
0 0 63 1
0 0 63 1
0 0 63 1
0 0 61 3
0 0 61 3
0 0 63 1
0 0 63 1
0 0 61 3
0 0 61 3
0 0 63 1
0 0 63 1
0 0 61 3
0 0 62 2
0 0 63 1
0 0 63 1
0 0 62 2
0 0 63 1
0 0 63 1
0 0 63 1
0 0 64 0
0 0 62 2
0 0 63 1
0 0 62 2
0 0 62 2
0 0 62 2
0 0 63 1
0 0 61 3
0 0 62 2
0 0 62 2
0 0 63 1
0 0 63 1
0 0 63 1
0 0 59 5
0 0 59 5
0 0 61 3
0 0 63 1
0 0 63 1
0 0 61 3
0 0 64 0
0 0 62 2
0 0 64 0
0 0 62 2
0 0 63 1
0 0 62 2
0 0 62 2
0 0 60 4
0 0 63 1
0 0 57 7
0 0 63 1
0 0 61 3
0 0 62 2
0 0 64 0
0 0 62 2
0 0 63 1
0 0 64 0
0 0 61 3
0 0 62 2
0 0 61 3
0 0 63 1
0 0 60 4
0 0 63 1
0 0 63 1
0 0 63 1
0 0 64 0
0 0 63 1
0 0 62 2
0 0 62 2
0 0 63 1
0 0 61 3
0 0 61 3
0 0 61 3
0 0 63 1
0 0 62 2
0 0 61 3
0 0 61 3
0 0 62 2
0 0 62 2
0 0 60 4
0 0 62 2
0 0 63 1
0 0 64 0
0 0 63 1
0 0 61 3
0 0 61 3
0 0 62 2
0 0 63 1
0 0 63 1
0 0 62 2
0 0 61 3
0

KeyboardInterrupt: 

In [96]:
test_inp = torch.from_numpy(np.array([0] * 64).reshape(1, 1, -1))
#test_inp[0,0,3] = 1
test_inp

tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [97]:
output = network(test_inp.float())

In [98]:
output.shape

torch.Size([1, 64])

In [99]:
output[output >= 0.5] = 1
output[output < 0.5] = 0
output

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 1.]], grad_fn=<IndexPutBackward0>)

In [45]:
class SRCNN(nn.Module):
    def __init__(self, window=WINDOW_SIZE):
        self.window = window
        super(SRCNN, self).__init__()
        self.layer1 = nn.Conv1d(window, window, kernel_size=1, stride=1, padding=0)
        self.layer2 = nn.Conv1d(window, 2 * window, kernel_size=1, stride=1, padding=0)
        self.fc1 = nn.Linear(2 * window, 4 * window)
        self.fc2 = nn.Linear(4 * window, window)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = x.view(x.size(0), self.window, 1)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return torch.sigmoid(x)

In [48]:
output2 = SRCNN(64)(test_inp.float())

In [49]:
output2.shape

torch.Size([1, 64])